In [1]:
import json
import pandas as pd
from collections import Counter
from collections import defaultdict as ddict
from matplotlib import pyplot as plt
import seaborn as sns
from tldextract import extract


pd.set_option('max_colwidth', 250)

In [2]:
domain_scan_report_pahth = '/export/sec02/nabeel/ucsc/Vaccine_Disinfo/data/covid_resources/oxford-vaccine-article-domains-shared-on-twitter.csv.scan.report'
domain_scan_report = list(map(json.loads,open(domain_scan_report_pahth,'r').readlines()))

In [3]:
extract('https://forbes.com.br/')

ExtractResult(subdomain='', domain='forbes', suffix='com.br')

In [4]:
def DF(df):
    data = []
    for i in df:
        try:
            domain = extract(i['url'])
            if domain.subdomain == '':
                data.append({'domain':'{}.{}'.format(domain.domain,domain.suffix),'positives':i['positives']})
            else:
                data.append({'domain':'{}.{}.{}'.format(domain.subdomain, domain.domain,domain.suffix),'positives':i['positives']})
        except:
            pass
    return pd.DataFrame(data)

In [5]:
vt = DF(domain_scan_report)

In [6]:
vt.head()

,domain,positives
0,forbes.com.br,0
1,google.com,1
2,xhne.ws,0
3,nbcnews.com,0
4,mol.im,0


In [7]:
vt.positives.unique()

array([0, 1, 2, 7, 5, 6, 3])

In [8]:
domain_scan_whois_pahth = '/export/sec02/nabeel/ucsc/Vaccine_Disinfo/data/covid_resources/oxford-vaccine-article-domains-shared-on-twitter.csv.whois'
domain_scan_whois = list(map(json.loads,open(domain_scan_whois_pahth,'r').readlines()))

In [9]:
def transform(x):
    for y in x:
        for i in y.get('data'):
            yield {
            'Domain Name':  i.get('attributes').get('whois_map').get('Domain Name'),
            'Creation Date':i.get('attributes').get('whois_map').get('Creation Date'),
            'expires':      i.get('attributes').get('whois_map').get('expires'),
            'Changed':      i.get('attributes').get('whois_map').get('Changed'),
            'Name Server':  i.get('attributes').get('whois_map').get('Name Server'),
            'nic-hdl-br':   i.get('attributes').get('whois_map').get('nic-hdl-br'),
            'status':       i.get('attributes').get('whois_map').get('status'),
            'registrant_country':i.get('attributes').get('registrant_country')
            }

In [10]:
whois = pd.DataFrame(list(transform(domain_scan_whois)))

In [11]:
whois.head()

,Domain Name,Creation Date,expires,Changed,Name Server,nic-hdl-br,status,registrant_country
0,None,None,None,None,None,None,None,None
1,forbes.com.br,19991201 #227065 | 20020619,20201201,20181011 | 20191031,leah.ns.cloudflare.com | stan.ns.cloudflare.com,NAB51,published,None
2,forbes.com.br,19991201 #227065 | 20020619,20201201,20181011 | 20191031,leah.ns.cloudflare.com | stan.ns.cloudflare.com,NAB51,published,BR
3,forbes.com.br,19991201 #227065 | 20020619,20191201,20181011 | 20181030,becky.ns.cloudflare.com | elliot.ns.cloudflare.com,NAB51,published,BR
4,forbes.com.br,19991201 #227065 | 20020619,20191201,20181011 | 20181030,becky.ns.cloudflare.com | elliot.ns.cloudflare.com,NAB51,published,None


In [12]:
DATA1 = ddict(set)
DATA2 = ddict(set)
data1 = ddict(list)
for index, row  in vt.iterrows():
    data = whois.loc[whois['Domain Name']==row.domain]
    if not data.empty:
        for i in data['Name Server']:
            try:
                nss = i.split('|')
                for y in nss:
                    ns = extract(y)
                    DATA1[row.positives].add('{}.{}'.format(ns.domain,ns.suffix))
                    DATA2[row.positives].add(y)
                    
                    data1[row.positives].append('{}.{}'.format(ns.domain,ns.suffix))
                    
                    
            except:
                pass
    else:
        DATA1[row.positives].add('NO-WHOIS')
        DATA2[row.positives].add('NO-WHOIS')
        
        data1[row.positives].append('NO-WHOIS')          

In [13]:
DF_DATA1 = ddict(list)
for i in sorted(DATA1.keys()):
    DF_DATA1['vt positives'].append(i)
    DF_DATA1['ns'].append(','.join(DATA1[i]))
DF_DATA1 = pd.DataFrame(DF_DATA1)

In [14]:
DF_DATA1

,vt positives,ns
0,0,"awsdns-02.com,name-services.com,pointerns.com,xadica.cat,srv53.net,domaincontrol.com,e-servidor.com,inet.it,exohosting.cz,dnsimple.com,formatsystem.it,uk.net,awsdns-42.co.uk,intendo.it,awsdns-14.co.uk,styria.com,servidorlatinoamerica.com,skynet.b..."
1,1,"awsdns-02.com,awsdns-53.org,domaincontrol.com,metrotvnews.com,cdmon.net,googledomains.com,arubadns.cz,criacaodesiteemsalvador.com.br,awsdns-24.net,arubadns.net,cloudflare.com,technorail.com,awsdns-34.net,awsdns-59.org,awsdns-35.net,gandi.net,dyne..."
2,2,"azure-dns.info,awsdns-41.co.uk,NO-WHOIS,domaincontrol.com,cloudflare.com,azure-dns.net,azure-dns.org,iq.pl,awsdns-58.net,hostwindsdns.com,awsdns-19.com,awsdns-07.org,hostthename.com,azure-dns.com"
3,3,NO-WHOIS
4,5,NO-WHOIS
5,6,NO-WHOIS
6,7,NO-WHOIS


In [15]:
DF_DATA2 = ddict(list)
for i in sorted(DATA2.keys()):
    DF_DATA2['vt positives'].append(i)
    DF_DATA2['ns'].append(','.join(DATA2[i]))
DF_DATA2 = pd.DataFrame(DF_DATA2)

In [16]:
DF_DATA2

,vt positives,ns
0,0,"ns2.cosmostelevision.pe, ns1.salesforce.com ,rudy.ns.cloudflare.com ,ns1.hostwebmedia.com.br , ns-1832.awsdns-37.co.uk , NS46.DOMAINCONTROL.COM ,NS2738.DIZINC.COM , ns-823.awsdns-38.net,donovan.ns.cloudflare.com ,conrad.ns.cloudflare.com , ns107..."
1,1,"ns-1454.awsdns-53.org ,ns-1372.awsdns-43.org ,ivan.ns.cloudflare.com , ns-20.awsdns-02.com , dns3.arubadns.net , ns-795.awsdns-35.net,borges.uol.com.br 200.147.255.105 , ns-cloud-a4.googledomains.com, ns3.p43.dynect.net , ns-1603.awsdns-08.co.uk..."
2,2,"ns2.iq.pl ,ns1-08.azure-dns.com , ns2-08.azure-dns.net , htnns8.hostthename.com,ns01.domaincontrol.com ,june.ns.cloudflare.com , ns-982.awsdns-58.net,carol.ns.cloudflare.com , ns02.domaincontrol.com, ns3-08.azure-dns.org , ruben.ns.cloudflare.co..."
3,3,NO-WHOIS
4,5,NO-WHOIS
5,6,NO-WHOIS
6,7,NO-WHOIS


In [17]:
DF_DATA1.to_csv('vt-apex-top.csv')
DF_DATA2.to_csv('vt-apex-sub.csv')

In [18]:
DF_data1 = ddict(list)
for i in sorted(data1.keys()):
    DF_data1['vt positives'].append(i)
    DF_data1['ns'].append(','.join(map(str,Counter(data1[i]).items())))
DF_data1 = pd.DataFrame(DF_data1)

In [19]:
DF_data1

,vt positives,ns
0,0,"('cloudflare.com', 952),('NO-WHOIS', 4371),('akam.net', 314),('DNSIMPLE.COM', 12),('dnsimple.com', 16),('parking.st', 6),('awsdns-11.org', 4),('awsdns-53.co.uk', 2),('awsdns-33.com', 19),('awsdns-25.net', 11),('ultradns.net', 54),('ultradns.org',..."
1,1,"('NO-WHOIS', 86),('nsone.net', 16),('dynect.net', 16),('awsdns-16.com', 4),('awsdns-53.org', 4),('awsdns-08.co.uk', 4),('awsdns-35.net', 4),('awsdns-43.org', 3),('awsdns-24.net', 3),('googledomains.com', 37),('cloudflare.com', 48),('awsdns-59.org..."
2,2,"('cloudflare.com', 18),('iq.pl', 6),('NO-WHOIS', 10),('awsdns-07.org', 1),('awsdns-19.com', 1),('awsdns-41.co.uk', 1),('awsdns-58.net', 1),('azure-dns.com', 3),('azure-dns.net', 3),('azure-dns.org', 3),('azure-dns.info', 3),('domaincontrol.com', ..."
3,3,"('NO-WHOIS', 3)"
4,5,"('NO-WHOIS', 1)"
5,6,"('NO-WHOIS', 1)"
6,7,"('NO-WHOIS', 1)"


In [20]:
DF_data1.to_csv('summary-vt-whois.csv')

In [21]:
vt.loc[vt.positives==3]

,domain,positives
1755,uttaranchaltoday.com,3
1860,trsondakika.com,3
2622,newssummedup.com,3


In [22]:
vt.loc[vt.positives==5]

,domain,positives
991,onerazz.com,5


In [23]:
vt.loc[vt.positives==6]

,domain,positives
1109,goo.su,6


In [24]:
vt.loc[vt.positives==7]

,domain,positives
104,gofirstnews.com,7
